In [43]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.support.ui import Select

In [44]:
df = pd.read_csv("ipl-2023.csv")
df.head()

,Unnamed: 0,Name,Matches,Runs,Batting Average,Wicket,Bowling Average
0,0,Abdul Samad,9,169,42.25,0,NaN
1,1,Abhishek Sharma,11,226,20.55,2,62.00
2,2,M A Agarwal,10,270,27.00,0,NaN
3,3,K K Ahmed,9,0,NaN,9,33.44
4,4,Akash Deep,2,17,17.00,1,59.00


In [45]:
# inplace
df.replace(np.nan, 0, inplace=True)
del df['Unnamed: 0']

In [46]:
df.head()

,Name,Matches,Runs,Batting Average,Wicket,Bowling Average
0,Abdul Samad,9,169,42.25,0,0.00
1,Abhishek Sharma,11,226,20.55,2,62.00
2,M A Agarwal,10,270,27.00,0,0.00
3,K K Ahmed,9,0,0.00,9,33.44
4,Akash Deep,2,17,17.00,1,59.00


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             209 non-null    object 
 1   Matches          209 non-null    int64  
 2   Runs             209 non-null    int64  
 3   Batting Average  209 non-null    float64
 4   Wicket           209 non-null    int64  
 5   Bowling Average  209 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 9.9+ KB


In [48]:
df = df.astype({"Matches":'float', "Runs":'float', "Batting Average":'float', "Wicket":'float', "Bowling Average":'float'}) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             209 non-null    object 
 1   Matches          209 non-null    float64
 2   Runs             209 non-null    float64
 3   Batting Average  209 non-null    float64
 4   Wicket           209 non-null    float64
 5   Bowling Average  209 non-null    float64
dtypes: float64(5), object(1)
memory usage: 9.9+ KB


In [49]:
# Define weightings for each parameter
matches_weight = 0.2
runs_weight = 0.3
batting_avg_weight = 0.15
wickets_weight = 0.2
bowling_avg_weight = 0.15

# Function to calculate player rating
def calculate_player_rating(matches, runs, batting_avg, wickets, bowling_avg):
    rating = (matches * matches_weight +
              runs * runs_weight +
              batting_avg * batting_avg_weight +
              wickets * wickets_weight +
              bowling_avg * bowling_avg_weight)
    return rating

In [50]:
# Iterate through each row and calculate player rating
ratings = []
for index, row in df.iterrows():
    rating = calculate_player_rating(row['Matches'], row['Runs'], row['Batting Average'], row['Wicket'], row['Bowling Average'])
    ratings.append(rating)

In [51]:
# Insert the "Rating" column into the DataFrame
df['Rating'] = ratings
df = df.astype({"Rating":'float'})
df.head()

,Name,Matches,Runs,Batting Average,Wicket,Bowling Average,Rating
0,Abdul Samad,9.0,169.0,42.25,0.0,0.00,58.8375
1,Abhishek Sharma,11.0,226.0,20.55,2.0,62.00,82.7825
2,M A Agarwal,10.0,270.0,27.00,0.0,0.00,87.0500
3,K K Ahmed,9.0,0.0,0.00,9.0,33.44,8.6160
4,Akash Deep,2.0,17.0,17.00,1.0,59.00,17.1000


In [52]:
# Finding the minimum and maximum rated player
maxi = max(df['Rating'])
mini = min(df['Rating'])
print(mini, maxi)

0.2 267.6185


In [53]:
# Normalising the Rating
n_rating = []
for index, row in df.iterrows():
    normalised_rating = ((row['Rating'] - mini)) / ((maxi - mini)) * 100
    n_rating.append(round(normalised_rating, 2))
df['Normalised Rating'] = n_rating
df.head()


,Name,Matches,Runs,Batting Average,Wicket,Bowling Average,Rating,Normalised Rating
0,Abdul Samad,9.0,169.0,42.25,0.0,0.00,58.8375,21.93
1,Abhishek Sharma,11.0,226.0,20.55,2.0,62.00,82.7825,30.88
2,M A Agarwal,10.0,270.0,27.00,0.0,0.00,87.0500,32.48
3,K K Ahmed,9.0,0.0,0.00,9.0,33.44,8.6160,3.15
4,Akash Deep,2.0,17.0,17.00,1.0,59.00,17.1000,6.32


In [54]:
df_2 = df.sort_values(['Normalised Rating'], ascending=False)
df_2 = df_2.drop('Rating', axis=1)
df_2.head()

,Name,Matches,Runs,Batting Average,Wicket,Bowling Average,Normalised Rating
172,Shubman Gill,16.0,851.0,60.79,0.0,0.0,100.00
41,F du Plessis,14.0,730.0,56.15,0.0,0.0,86.02
77,V Kohli,14.0,639.0,53.25,0.0,0.0,75.64
29,D P Conway,15.0,625.0,52.08,0.0,0.0,74.08
65,Y B K Jaiswal,14.0,625.0,48.08,0.0,0.0,73.78
